In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import Isomap
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_excel(r"all_samples_clean.xlsx", sheet_name="Sheet1")
data = data.drop(['BASIC_entity_name', 'BASIC_year'], axis=1)

#将非0值转换为1
data = data.applymap(lambda x: 1 if x != 0 else 0)

In [6]:
data.shape

(10536, 41)

In [14]:
# 创建Isomap对象，设置n_neighbors参数（近邻数）和n_components参数（输出维度数）
n_neighbors = 10  # 近邻数，可以根据数据集大小适当调整
n_components = 10  # 输出维度数，通常设置为2或3便于可视化

In [15]:
isomap = Isomap(n_neighbors=n_neighbors, n_components=n_components)

In [16]:
# 对数据进行拟合和转换
transformed_data = isomap.fit_transform(data)

In [17]:
transformed_data

array([[-1.87334512,  1.56516357, -3.00442183, ..., -0.01108428,
        -1.63583291,  0.07418814],
       [-2.46830192,  0.71597329, -1.72873636, ..., -2.00807046,
        -0.9170536 ,  1.95030761],
       [-3.29961682,  2.26476896,  1.73982038, ...,  0.19890005,
        -0.33721885,  0.65497481],
       ...,
       [ 1.1780642 , -1.00919148,  0.89635939, ...,  3.45924433,
         2.05838436, -1.4502577 ],
       [-0.37365202,  1.67685653, -1.00619739, ...,  0.64032375,
         3.71471197, -1.46248627],
       [-0.41503599,  3.4809088 , -2.8282477 , ...,  0.3532993 ,
         0.83361434,  1.20321937]])

In [21]:
# 使用k-neighbors方法计算重构误差
k_neighbors = 5  # 可根据实际情况调整k的值

# 找到映射后的数据中每个点的k个最近邻
nn = NearestNeighbors(n_neighbors=k_neighbors)
nn.fit(transformed_data)
distances, indices = nn.kneighbors(transformed_data)

# 近似重构误差
reconstruction_error = np.mean(distances)


In [23]:
reconstruction_error

1.5712938811758015

In [27]:
n_neighbors_list = [5, 10, 15, 20, 25]
# 使用交叉验证找到最优的k_neighbors值
for k_neighbors in n_neighbors_list:
    isomap = Isomap(n_neighbors=k_neighbors, n_components=n_components)
    pipeline = Pipeline([('isomap', isomap)])
    scores = cross_val_score(isomap, data, cv=5)  # 使用5折交叉验证
    mean_score = np.mean(scores)
    print(f"k_neighbors={k_neighbors}, Mean Score={mean_score}")
    
    # 保存最优的k_neighbors值和对应的评分
    if mean_score > best_mean_score:
        best_mean_score = mean_score
        best_k_neighbors = k_neighbors

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator Isomap(n_components=10) does not.